<a href="https://colab.research.google.com/github/kiruthika514/.-Astronaut-Daily-Schedule-Organizer-Programming-Exercise/blob/main/sample-lstm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow pandas numpy h5py


In [2]:
from numpy import array
from pickle import load,dump
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import *
from tensorflow.keras.layers import Add
from keras.callbacks import ModelCheckpoint
import pydot
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split(' ')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
    # load all features
    all_features = load(open(filename, 'rb'))
    #print(all_features)
    # filter features
    features = {k: all_features[k] for k in dataset}
    return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# create sequences of images, input sequences and output words for an image
def create_sequences(domain, tokenizer, max_length, desc_list, photo):
    X1, X2, X3, y = list(), list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(photo)
            X2.append(domain)
            X3.append(in_seq)
            y.append(out_seq)
    return array(X1), array(X2), array(X3), array(y)

#data generator, intended to be used in a call to model.fit_generator()
def data_generator(domain_features, descriptions, photos, tokenizer, max_length):
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            # retrieve the photo feature
            domain = domain_features[key]
            photo = photos[key][0]
            in_img, in_domain, in_seq, out_word = create_sequences(domain, tokenizer, max_length, desc_list, photo)
            yield [[in_img, in_domain, in_seq], out_word]

# load training dataset (6K)
filename = '/content/drive/MyDrive/finalattention/Description/trainuc.txt'
train = load_set(filename)
#print(sorted(train))
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/drive/MyDrive/finalattention/Description/trainuc.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
#print(train)
#all_features = load(open('features_inceptionv3_uc.pkl', 'rb'))
#print(all_features)

train_features = load_photo_features('/content/drive/MyDrive/finalattention/Features/features_uc_resnet152_updated.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
#dump(tokenizer,open('tokenizer_resnet152.pkl','wb'))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
## determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)


Dataset: 1260
Descriptions: train=1260
Photos: train=1260
Vocabulary Size: 293
Description Length: 24


In [5]:
features=load(open('/content/drive/MyDrive/finalattention/Features/features_resnet152_newids.pkl','rb'))
keys  = features.keys()
l = list()
for i in train:
    if i not in keys:
        #print(i)
        l.append(i)
#print(sorted(l))
print(len(l))

1260


In [6]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(domain, model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        domain1 = np.array([domain])
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,domain1,sequence], verbose=0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

# evaluate the skill of the model
def evaluate_model(domain, model, descriptions, photos, tokenizer, max_length,filename):
    actual, predicted = list(), list()
    # step over the whole set
    lines = list()
    for key, desc_list in descriptions.items():
        yhat = generate_desc(domain[key], model, tokenizer, photos[key], max_length)
        ex=yhat
        a=yhat.split('startseq')
        b=a[1].split('endseq')
        lines.append('beam_size_1'+'\t'+key + '\t' + b[0])
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
        #
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    bleu=np.zeros(4)
    # calculate BLEU score
    bleu[0]=corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
    bleu[1]=corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
    bleu[2]=corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
    bleu[3]=corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))
    print('BLEU-1: %f' % bleu[0])
    print('BLEU-2: %f' % bleu[1])
    print('BLEU-3: %f' % bleu[2])
    print('BLEU-4: %f' % bleu[3])
    return bleu

In [7]:
# load test set
filename = '/content/drive/MyDrive/finalattention/Description/testuc.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/content/drive/MyDrive/finalattention/Description/testuc.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
#print(test)
test_features = load_photo_features('/content/drive/MyDrive/finalattention/Features/features_uc_resnet152_updated.pkl', test)
print('Photos: test=%d' % len(test_features))

Dataset: 420
Descriptions: test=420
Photos: test=420


In [8]:
features = load(open('/content/drive/MyDrive/finalattention/Features/features_resnet152_newids.pkl','rb'))
keys = features.keys()
for i in test:
    if i not in keys:
        print(i)

04_070
16_078
19_064
15_061
10_068
03_068
17_069
18_063
17_080
04_071
07_067
07_068
11_073
07_077
02_077
12_068
02_063
06_079
15_070
11_068
17_075
07_076
14_070
01_070
10_079
05_076
16_077
20_071
07_063
15_067
02_075
05_078
03_076
10_077
08_079
00_065
06_070
09_077
17_061
20_068
14_080
16_067
20_070
17_071
06_068
06_063
04_062
07_069
20_075
19_070
04_069
12_071
12_074
00_077
10_063
06_075
01_063
14_064
16_065
07_064
01_067
16_061
14_074
10_073
00_074
18_064
05_074
17_068
17_070
18_078
17_073
06_069
15_066
20_063
11_065
15_076
05_064
15_062
15_077
06_071
10_080
08_068
20_072
10_078
00_070
13_075
11_071
17_077
14_079
04_076
17_063
20_067
10_067
06_076
10_065
02_066
03_064
18_075
07_066
10_061
02_072
12_072
01_071
16_064
00_062
07_079
11_078
14_073
04_065
05_072
19_079
04_061
14_075
15_078
20_080
08_061
09_079
06_080
00_066
02_080
20_066
05_069
12_080
06_077
05_070
08_069
18_076
10_066
05_073
02_065
04_080
20_062
10_071
12_066
09_073
16_063
18_077
09_078
19_063
13_069
12_070
08_077
02_070

In [9]:
!pip install tensorflow


In [10]:
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim


In [11]:
!pip install gensim


In [14]:
import gensim
from gensim.models import Word2Vec
from pickle import dump

# Load the dataset (modify this based on your dataset file)
dataset_path = "/content/drive/MyDrive/finalattention/Description/trainnew.txt"

# Read captions from dataset
with open(dataset_path, "r") as f:
    captions = f.readlines()

# Preprocess: Convert to lowercase, split into words
sentences = [caption.strip().lower().split() for caption in captions]

# Train a Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)

# Save the embedding as a dictionary
word_vectors = {word: word2vec_model.wv[word] for word in word2vec_model.wv.index_to_key}

# Save to a file
embedding_path = "/content/drive/MyDrive/finalattention/Features/word2vec_embedding.pkl"
with open(embedding_path, "wb") as f:
    dump(word_vectors, f)

print("✅ Word2Vec Embedding saved successfully!")


✅ Word2Vec Embedding saved successfully!


In [15]:
# load a word embedding
def load_embedding(tokenizer, vocab_size, max_length):
	# load the tokenizer
	embedding = load(open('/content/drive/MyDrive/finalattention/Features/word2vec_embedding.pkl', 'rb'))
	dimensions = 100
	trainable = False
	# create a weight matrix for words in training docs
	weights = np.zeros((vocab_size, dimensions))
	# walk words in order of tokenizer vocab to ensure vectors are in the right index
	for word, i in tokenizer.word_index.items():
		if word not in embedding:
			continue
		weights[i] = embedding[word]
	layer = Embedding(vocab_size, dimensions, weights=[weights], input_length=max_length, trainable=trainable, mask_zero=True)
	return layer

In [16]:
import os

# Define the path
model_dir = "/content/drive/MyDrive/finalattention/Modelcall"

# Create the directory if it does not exist
os.makedirs(model_dir, exist_ok=True)

# Now define the model
model = define_model(vocab_size, max_length)


NameError: name 'define_model' is not defined

In [ ]:
# define the captioning model
loc = '/content/drive/MyDrive/finalattention/Modelcall/'
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(2048,))
    #fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(inputs1)

    inputs2 = Input(shape=(21,))
    merged = concatenate([fe2,inputs2])

    fe3 = RepeatVector(max_length)(merged)

    # sequence model
    inputs3 = Input(shape=(max_length,))
    #se1 = load_embedding(tokenizer, vocab_size, max_length)(inputs3)
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs3)
    #print(se1.shape)
    se2 = LSTM(256,return_sequences=True)(se1)
    #print(se2.shape)
    #x = Attention(max_length)(se2)
    #print(x.shape)
    se3 = TimeDistributed(Dense(256,activation='relu'))(se2)
    #print(se3.shape)

    # decoder model
    decoder1 = concatenate([fe3, se3])
    decoder2 = Bidirectional(LSTM(150,return_sequences=True))(decoder1)
    decoder3 = Attention(max_length)(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    model.summary()
    plot_model(model, to_file=loc+'model.png', show_shapes=True)
    return model

In [ ]:
model = define_model(vocab_size, max_length)

In [ ]:
embedding = load(open('UCMDataset1/Features/word2vec_embedding.pkl', 'rb'))
embedding

In [ ]:
domain_features = load(open('drive/My Drive/Final Year Project/Features/domain_features.pkl', 'rb'))
print(domain_features['100_077'].shape)

In [ ]:
## train the model, run epochs manually and save after each epoch
epochs = 20
steps = len(train_descriptions)
mylist = list()
df=pd.DataFrame(index=mylist, columns=['model_no','Bleu_1','Bleu_2','Bleu_3','Bleu_4'])
for i in range(1,epochs):
    print(i)
    # create the data generator
    generator = data_generator(domain_features, train_descriptions, train_features, tokenizer, max_length)
    # fit for one epoch
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # save model
    model.save(loc+'model_' + str(i) + '.h5')
    # load the mode
    filename = loc+'result/text_predicted_' + str(i) + '.txt'
    # evaluate model
    bleu = evaluate_model(domain_features, model, test_descriptions, test_features, tokenizer, max_length,filename)
    #bleu = evaluate_model(model, test_descriptions, test_features,tokenizer, max_length,filename_new)
    df.at[i,'model_no']=i
    df.at[i,'Bleu_1']=bleu[0]
    df.at[i,'Bleu_2']=bleu[1]
    df.at[i,'Bleu_3']=bleu[2]
    df.at[i,'Bleu_4']=bleu[3]
    df.to_csv(loc+'result/results.csv', index=False)

In [ ]:
## from keras.models import load_model
filename = 'UCMDataset/text_predicted.txt'
model = model.load_weights('Modelnewdataall/model_1.h5')
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length,filename)